In [1]:
import pandas as pd
import numpy as np

In [2]:
def mpNumCoEvents(closeIdx, t1, molecule):
    """
    Compute the number of concurrent events per bar.
    + molecule[0] is the date of the first event on which the weight will be computed
    + molecule[-1] is the date of the last event on which the weight will be computed 
    Any event that starts before t1[molecule].max() impacts the count.
    """
    # 1) find events that span the period [molecule[0], molecule[-1]]
    t1 = t1.fillna(closeIdx[-1])                 # unclosed events still must impact other weights
    t1 = t1[t1 >= molecule[0]]                   # events that end at or after molecule[0]
    t1 = t1.loc[:t1[molecule].max()]             # events that start at or before t1[molecule].max()

    # 2) count events spanning a bar
    iloc = closeIdx.searchsorted(np.array([t1.index[0], t1.max()]))
    count = pd.Series(0, index=closeIdx[iloc[0]:iloc[1] + 1])
    for tIn in t1.index.tolist():
        tOut = t1.loc[tIn]
        count.loc[tIn:tOut] += 1.0

    return count.loc[molecule[0]:t1[molecule].max()]

In [3]:
# Sample close index (e.g. daily timestamps)
closeIdx = pd.date_range('2020-01-01', periods=10, freq='D')

# Sample t1: start and end times of each event
# Think of each index as event start (t_in) and its value as event end (t_out)
t1 = pd.Series({
    pd.Timestamp('2020-01-01'): pd.Timestamp('2020-01-04'),
    pd.Timestamp('2020-01-02'): pd.Timestamp('2020-01-05'),
    pd.Timestamp('2020-01-03'): pd.Timestamp('2020-01-06'),
    pd.Timestamp('2020-01-05'): pd.Timestamp('2020-01-07'),
    pd.Timestamp('2020-01-08'): pd.NaT,  # open-ended event
})

# Molecule = subset of events for multiprocessing (can be a slice)
molecule = t1.index[1:4]   # events starting on 2020-01-02, 2020-01-03, 2020-01-05

In [4]:
count = mpNumCoEvents(closeIdx, t1, molecule)
count

2020-01-02    2
2020-01-03    3
2020-01-04    3
2020-01-05    3
2020-01-06    2
2020-01-07    1
Freq: D, dtype: int64

In [5]:
def mpSampleTW(t1, numCoEvents, molecule):
    # Derive average uniqueness over the event's lifespan
    wght = pd.Series(index=molecule)
    for tIn in molecule:
        tOut = t1.loc[tIn] 
        wght.loc[tIn] = (1.0 / numCoEvents.loc[tIn:tOut]).mean()
    return wght

In [6]:
wght = mpSampleTW(t1, count, molecule)
wght

2020-01-02    0.375000
2020-01-03    0.375000
2020-01-05    0.611111
dtype: float64

In [7]:
def getIndMatrix(barIx, t1):
    # get indication matrix
    indM = pd.DataFrame(0, index=barIx, columns=range(t1.shape[0]))
    for i, tIn in enumerate(t1.index):
        tOut = t1.loc[tIn]
        indM.loc[tIn:tOut, i] = 1
    return indM

In [8]:
indM = getIndMatrix(closeIdx, t1)
indM

,0,1,2,3,4
2020-01-01,1,0,0,0,0
2020-01-02,1,1,0,0,0
2020-01-03,1,1,1,0,0
2020-01-04,1,1,1,0,0
2020-01-05,0,1,1,1,0
2020-01-06,0,0,1,1,0
2020-01-07,0,0,0,1,0
2020-01-08,0,0,0,0,1
2020-01-09,0,0,0,0,1
2020-01-10,0,0,0,0,1


In [9]:
def getAvgUniq(indM):
    # Average Uniqueness from Indication Matrix
    c = indM.sum(axis=1) # concurrency
    u = indM.div(c, axis=0) # uniqueness
    avgU = u[u>0].mean() # average uniqueness
    return avgU

In [10]:
avgU = getAvgUniq(indM)
avgU

0    0.541667
1    0.375000
2    0.375000
3    0.611111
4    1.000000
dtype: float64

In [11]:
def seqBootstrap(indM, sLength=None):
    # generate a sample via sequential bootstrap
    if sLength is None: sLength=indM.shape[1]
    phi = []
    while len(phi) < sLength:
        avgU = pd.Series()
        for i in indM:
            indM_ = indM[phi+[i]]
            avgU.loc[i] = getAvgUniq(indM_).iloc[-1]
        prob = avgU / avgU.sum() # draw prob
        phi += [np.random.choice(indM.columns, p=prob)]
    return phi

In [12]:
# def main():
#     t1=pd.Series([2,3,5],index=[0,2,4]) # t0,t1 for each feature obs
#     barIx=range(t1.max()+1) # index of bars
#     indM=getIndMatrix(barIx,t1)
#     phi=np.random.choice(indM.columns,size=indM.shape[1])
#     print(phi)
#     print('Standard uniqueness:',getAvgUniq(indM[phi]).mean())
#     phi=seqBootstrap(indM)
#     print(phi)
#     print('Sequential uniqueness:',getAvgUniq(indM[phi]).mean())
#     return

In [13]:
# main()

In [14]:
def getRndT1(numObs, numBars, maxH):
    # random t1 series 
    t1 = pd.Series()
    for i in range(numObs):
        ix = np.random.randint(0, numBars)
        val = ix + np.random.randint(1, maxH)
        t1.loc[ix] = val
    return t1.sort_index()

In [15]:
def getRndT1New(numObs=1000, numBars=10000, maxH=1000):
    """
    Generate a random t1 Series representing event end times.
    Each index represents the start of an event, and each value its end.
    """
    # Random start times
    t0 = np.random.randint(0, numBars, numObs)
    # Random holding period
    t1 = t0 + np.random.randint(1, maxH, numObs)
    # Cap values at numBars
    t1 = np.minimum(t1, numBars)
    # Make pandas Series
    t1 = pd.Series(t1, index=t0)
    # Sort by index
    t1 = t1[~t1.index.duplicated(keep='first')].sort_index()
    return t1

In [16]:
t1Old, t1New = getRndT1(1000, 10000, 1000), getRndT1New(1000, 10000, 1000)

In [17]:
def auxMC(numObs,numBars,maxH):
    # Parallelized auxiliary function
    t1=getRndT1New(numObs,numBars,maxH)
    barIx=range(t1.max()+1)
    indM=getIndMatrix(barIx,t1)
    phi=np.random.choice(indM.columns,size=indM.shape[1])
    stdU=getAvgUniq(indM[phi]).mean()
    phi=seqBootstrap(indM)
    seqU=getAvgUniq(indM[phi]).mean()
    return {'stdU':stdU,'seqU':seqU}

In [24]:
def mpSampleW(t1, numCoEvents, close, molecule):
    # Derive sample weights by return attribution
    ret=np.log(close).diff()
    wght=pd.Series(index=molecule)
    for tIn in molecule:
        tOut=t1.loc[tIn]
        wght.loc[tIn]=(ret.loc[tIn:tOut]/numCoEvents.loc[tIn:tOut]).sum()
    return wght.abs()

In [25]:
close = pd.Series(
    [100, 102, 101, 103, 105, 104, 106, 108, 107, 109],
    index=pd.date_range('2020-01-01', periods=10, freq='D'))

wght = mpSampleW(t1, count, close, t1.index)
wght

2020-01-01    0.013153
2020-01-02    0.019564
2020-01-03    0.004878
2020-01-05    0.020674
2020-01-08    0.000000
dtype: float64

In [41]:
def getTimeDecay(tW, clfLastW=1):
    # apply piecewise linear time decay to observerd uniqueness (tW)
    # newest observations get weight = 1, oldest pbservation get weight = clfLastW
    clfW = tW.sort_index().cumsum()
    if clfLastW>=0: slope=(1.-clfLastW)/clfW.iloc[-1]
    else: slope=1./ ((clfLastW+1)*clfW.iloc[-1])
    const=1.-slope*clfW.iloc[-1]
    clfW=const + slope*clfW
    clfW[clfW<0]=0
    print(const, slope)
    return clfW

In [48]:
clfW = getTimeDecay(wght, clfLastW=-.5)
clfW

-0.9999999999999998 34.32362239815314


2020-01-01    0.000000
2020-01-02    0.122973
2020-01-03    0.290396
2020-01-05    1.000000
2020-01-08    1.000000
dtype: float64